# Continuons avec Pandas

Les premières lignes : charger des données

In [1]:
import pandas as pd
url = "https://raw.githubusercontent.com/OpenAPC/openapc-de/master/data/snsf/snsf_openapc.csv"
corpus = pd.read_csv(url)

In [4]:
corpus.columns

Index(['institution', 'period', 'euro', 'doi', 'journal_full_title',
       'publisher', 'is_hybrid', 'issn', 'url', 'chf', 'conversion_rate',
       'invoice_original_amount', 'invoice_original_currency',
       'snsf_voucher_number', 'grant_snsf', 'grant_snsf_related'],
      dtype='object')

## Fonctions d'accompagnement du traitement de données

Gestion des valeurs nulles

In [6]:
pd.isnull(corpus["euro"]).value_counts()

False    718
Name: euro, dtype: int64

In [7]:
pd.isnull(corpus["doi"]).value_counts()

False    717
True       1
Name: doi, dtype: int64

Enlever toutes les lignes où il y a au moins une valeur absente (None)

In [15]:
#corpus[["euro","period"]].dropna()
corpus[["euro","period"]].fillna("Valeur nulle")

,euro,period
0,896.45,2011
1,929.92,2012
2,1784.42,2012
3,972.18,2012
4,1254.45,2012
...,...,...
713,1498.00,2019
714,369.55,2019
715,1256.28,2019
716,2269.33,2019


### La phase de recodage des données

In [17]:
corpus["Nouvelle colonne"] = 1

In [21]:
corpus["journal_full_title_lower"] = corpus["journal_full_title"].str.lower()

In [22]:
len(corpus["journal_full_title_lower"].unique())

269

In [24]:
len(corpus["journal_full_title"].unique())

273

Comment faire pour recoder élément par élément une colonne d'un tableau en suivant une règle.

Recodage d'une variable numérique en variable par catégorie

Règle générale :

1. Définir une fonction qui fait l'opération sur un élément
2. On va l'appliquer sur chacun des éléments d'une colonne

Pour recoder une variable numérique : définir une fonction qui prend en entrée un nombre et qui renvoie la catégorie associée souhaitée (moins de 1000, entre 1000 et 3000, entre 3000 et 4000 et plus de 4000)

Etape 1 : définir la fonction qui fait l'opération sur un cas

In [43]:
def recodage(case):
    
    if pd.isnull(case):
        return None
    
    if case < 1000:
        return "1-Moins de 1000"
    elif case < 3000:
        return "2-Entre 1000 et 3000"
    elif case < 4000:
        return "3-Entre 3000 et 4000"
    else:
        return "4-Plus de 4000"

In [30]:
recodage(10)

'1-Moins de 1000e'

Etape 2 : l'appliquer sur une colonne avec la méthode .apply

In [44]:
corpus["invoice_recoded"] = corpus["invoice_original_amount"].apply(recodage)

In [45]:
corpus["invoice_recoded"].value_counts()

2-Entre 1000 et 3000    392
4-Plus de 4000           74
1-Moins de 1000          71
3-Entre 3000 et 4000     68
Name: invoice_recoded, dtype: int64

Exercice : créer une nouvelle colonne avec le nom des journaux sans espaces et en minuscule


1. on définit une fonction qui prend en entrée un texte, et renvoie le texte sans les espaces en minuscule
2. j'applique cette fonction à la colonne journal_full_title

In [52]:
def recodage_texte(titre):
    # mettre en minuscule
    titre = titre.lower()
    # remplacer les espaces
    titre = titre.replace(" ","")
    return titre

#def recodage_texte(titre):
#    return titre.lower().replace(" ","")

In [49]:
recodage_texte("Ceci est un titre")

'ceciestuntitre'

In [53]:
len(corpus["journal_full_title"].apply(recodage_texte).unique())

269

### Les fonctions anonymes

Faire la fonction qui multiplie un nombre par 10

In [55]:
(lambda x : 10*x)(100)

1000

In [56]:
corpus["journal_full_title"].apply(lambda x: x.lower().replace(" ",""))

0                                      plosone
1                plosneglectedtropicaldiseases
2                                plospathogens
3                      frontiersinmicrobiology
4      internationaljournalofmolecularsciences
                        ...                   
713                              thecryosphere
714              journalofeducationandlearning
715                          ecologyandsociety
716                                      elife
717                                      peerj
Name: journal_full_title, Length: 718, dtype: object

Si on veut faire une boucle sur chaque ligne d'un tableau

In [61]:
for i,ligne in corpus.iterrows(): # c'est un itérateur qui renvoie deux éléments : l'index et la ligne
    if "Cell" in ligne["journal_full_title"]:
        print(ligne["journal_full_title"])

Cellular Physiology and Biochemistry
Cell Reports
Cell Reports
Cell Reports
Cell Discovery
Frontiers in Cellular and Infection Microbiology
Cell Reports
Frontiers in Cell and Developmental Biology
Frontiers in Cellular and Infection Microbiology
Plant Cell, Tissue and Organ Culture (PCTOC)
Cell Reports
Cell Reports
Molecular Cell
European Cells and Materials
Frontiers in Cellular Neuroscience
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Stem Cell Reports
Cell Death & Disease
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Cell Reports
Stem Cell Reports
Frontiers in Cellular Neuroscience


On peut regrouper les éléments d'un tableau par rapport à une colonne

Je veux calculer les dépenses par année du FNS

In [64]:
corpus.groupby("period")["euro"].sum()

period
2011       896.45
2012      4940.97
2013     26488.29
2014     71904.28
2015    175968.53
2016    193439.05
2017    139668.57
2018    376130.45
2019    449944.69
Name: euro, dtype: float64

In [66]:
# grouper par la colonne period, prendre la colonne euro de chaque élément, 
# appliquer un ensemble de fonctions avec agg
corpus.groupby("period")["euro"].agg(["sum","median","mean","std"])

,sum,median,mean,std
period,,,,
2011,896.45,896.450,896.450000,NaN
2012,4940.97,1113.315,1235.242500,393.441466
2013,26488.29,1115.945,1204.013182,414.835939
2014,71904.28,1092.330,1331.560741,612.570624
2015,175968.53,1389.295,1795.597245,1025.868529
2016,193439.05,1543.945,1824.896698,877.518207
2017,139668.57,1720.090,1967.162958,991.153982
2018,376130.45,1937.010,2307.548773,1192.963623
2019,449944.69,1785.890,2261.028593,1270.304621


In [72]:
corpus.groupby("journal_full_title")["euro"].agg(["count","sum","median","mean","std"]).sort_values("count")

,count,sum,median,mean,std
journal_full_title,,,,,
AAPS Open,1,1929.55,1929.550,1929.550000,NaN
Journal of Maps,1,506.68,506.680,506.680000,NaN
Journal of Medical Internet Research,1,2267.56,2267.560,2267.560000,NaN
Journal of Molecular Biology,1,1368.78,1368.780,1368.780000,NaN
Journal of Neurochemistry,1,2724.00,2724.000,2724.000000,NaN
...,...,...,...,...,...
Frontiers in Psychology,19,30646.17,1413.970,1612.956316,639.346650
Cell Reports,24,104638.36,4430.190,4359.931667,446.033215
PLOS ONE,46,61316.44,1321.745,1332.966087,98.239324
